In [1129]:
import cv2
import numpy as np

all_outputs = True
input_file = '12003-2.jpg'
img = cv2.imread(input_file) #img = 3D array, with [row, col, b/g/r channel]

In [1130]:
# Blur the image, remove noise
img_blur = cv2.bilateralFilter(img, 30, 160, 160)

In [1131]:
kernel = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])

# Detect edges
img_filtered = cv2.filter2D(img_blur, -1, kernel)
img_filtered = cv2.cvtColor(img_filtered, cv2.COLOR_BGR2GRAY)

In [1132]:
img_hsv = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
img_hue, img_sat, img_val = cv2.split(img_hsv)

img_val_filtered = cv2.filter2D(img_val, -1, kernel)
img_hue_filtered = cv2.filter2D(img_hue, -1, kernel)
img_sat_filtered = cv2.filter2D(img_sat, -1, kernel)

# Combine multiple results to capture more edges
img_combined = img_val_filtered + img_hue_filtered + img_filtered + img_sat_filtered

# Reduce noise
k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
img_closed = cv2.morphologyEx(img_combined, cv2.MORPH_CLOSE, k, iterations=2)

# Invert to be black on white and increase contrast
img_final = cv2.bitwise_not(img_closed)
img_final_manual = img_final.copy()
for row in img_final_manual:
    for i, x in enumerate(row):
        if x > 200:
            row[i] = 255
        elif x <= 200 and x > 100:
            row[i] -= 10
cv2.imwrite("output/final.jpg",img_final)
cv2.imwrite("output/final_manual.jpg",img_final_manual)

# Write all steps in the middle
if all_outputs:
    cv2.imwrite("output/img_blur.jpg", img_blur)
    cv2.imwrite("output/img_sat.jpg", img_sat_filtered)
    cv2.imwrite("output/img_hue.jpg", img_hue_filtered)
    cv2.imwrite("output/img_val.jpg", img_val_filtered)
    cv2.imwrite("output/img_combined.jpg", img_combined)
    cv2.imwrite("output/img_closed.jpg", img_closed)

In [1133]:
#cv2.imshow("Final Output", img_final)
#cv2.waitKey(0)
#cv2.destroyAllWindows()